<a href="https://colab.research.google.com/github/iAmanSharan/Sentiment-analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle

In [ ]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()



Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 70% 57.0M/80.9M [00:00<00:00, 179MB/s]
100% 80.9M/80.9M [00:00<00:00, 186MB/s]


In [ ]:
import zipfile
with zipfile.ZipFile('sentiment140.zip', 'r') as zip_ref:
    zip_ref.extractall('path/to/extract')

In [ ]:
import pandas as pd
df = pd.read_csv("/content/path/to/extract/training.1600000.processed.noemoticon.csv",encoding='latin-1')
print(df.head())

In [10]:
df.columns = ['score' , 'a','b','c','d',"tweets"]

In [11]:
cols_name = df.columns.tolist()

In [ ]:
print(df.head())

In [16]:
from sklearn.model_selection import train_test_split

# Assuming your data is stored in a DataFrame called 'df'
X = df['tweets']
y = df['score']

# Split the data into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Split the testing set further into testing and validation sets (50% testing, 50% validation)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM , Embedding
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np

max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)

Xtrain_sequence = tokenizer.texts_to_sequences(X_train)
Xtest_sequence = tokenizer.texts_to_sequences(X_test)
Xvalidation_sequence = tokenizer.texts_to_sequences(X_val)

Xtrain_pad_seq = pad_sequences(Xtrain_sequence,maxlen = max_len)
Xtest_pad_seq = pad_sequences(Xtest_sequence,maxlen = max_len)
Xvalidation_pad_seq = pad_sequences(Xvalidation_sequence,maxlen = max_len)

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
y_val_cat = to_categorical(y_val)

model = Sequential()
model.add(Embedding(max_words,128))
model.add(LSTM(128))
model.add(Dense(3,activation='softmax'))

Xtrain_sequence = np.asarray(Xtrain_sequence)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(Xtrain_sequence, y_train_cat, validation_data=(Xvalidation_pad_seq , y_val_cat), epochs=10)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()